# TP en Traitement Automatique du Langage Naturel: Classification de Sentiments sur des Critiques de Films

## Objectif
L'objectif de ce TP est de développer un système de classification de sentiments utilisant des critiques de films. Vous utiliserez un ensemble de données IMDb et appliqueront un modèle K-Nearest Neighbors (KNN) pour classer les critiques en catégories positives ou négatives.

## Partie 1: Traitement des Textes
1. **Preprocess**: Appliquer un preprocess si besoin
1. **Vectorisation**: Transformez les documents textuels en vecteurs numériques en utilisant `TfidfVectorizer`.

## Partie 3: Modélisation
1. **Construction du Modèle KNN**: Créez un modèle KNN
2. **Entraînement du Modèle**: Entraînez le modèle sur l'ensemble d'entraînement.

## Partie 4: Évaluation
1. **Prédiction et Classification**: Utilisez le modèle pour prédire les sentiments sur l'ensemble de test.
2. **Rapport de Classification**: Générez un rapport de classification pour évaluer la performance du modèle.

## Questions
1. Comment la réduction du nombre de caractéristiques (`max_features`) affecte-t-elle la performance du modèle ?
2. Quel impact a le choix du nombre de voisins dans KNN sur les résultats ?
3. Comparez les performances du modèle KNN avec un autre classificateur (par exemple, [Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) ou [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC). Lequel performe mieux et pourquoi ?
4. Le preprocessing améliore t-il la clasification ?

## Ressources
- [IMDb Dataset](https://huggingface.co/datasets/imdb)
- [Scikit-learn Documentation](https://scikit-learn.org/stable/)

In [15]:
# !pip install -q -U datasets scikit-learn
# !pip install matplotlib seaborn spacy
# !python -m spacy download en_core_web_sm
# !pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 1.9 MB/s eta 0:00:00m eta 0:00:010:00:01m


## Chargement du dataset

In [35]:
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# On split plusieur fois le dataset afin de réduire le temps de calcule
dataset = load_dataset("imdb", split="train")
dataset = dataset.train_test_split(stratify_by_column="label", test_size=0.3, seed=42)
dataset = dataset['train'].train_test_split(stratify_by_column="label", test_size=0.3, seed=42)

## Visualisation du dataset

In [36]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 12250
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5250
    })
})

In [37]:
dataset['train']['text'][:5]

['I quote Oedpius Rex because it is a tragedy that this film was even made!!!<br /><br />This is one of the worst movies I have ever seen! I am in no way an Uwe Boll hater like most of the humourless people on IMDb! <br /><br />Uwe Boll movies like Postal and Tunnel Rats are hilariously bad and therefore entertaining. But honestly, this movie was just horrible. I hated it so much that I\'d give it a zero star rating if I could. The story is just crap! It spends four fifths of the film building the plot and then they have the middle which is just scenes of grizzly horrible tastelessly done murder! The finally end it with a "villan wins ending" which is totally acceptable but surely it could have been more tasteful than this! <br /><br />I am not against Uwe Boll (like I said earlier) nor am I against violent movies! I f**king love violent movies! I loved the Saw movies, the Hostel movies, Tokyo Gore Police, The New York Ripper, the 28 movies, Dog Soldiers, My Bloody Valentine, Last Hous

In [38]:
dataset['train']['label'][:5]
# 0 = negatif, 1 = positif

[0, 0, 1, 0, 0]

On remarque que le dataset contients des critiques de films, le label correspond a si la critique est positive ou négative

In [39]:
import plotly.graph_objects as go
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd


def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n]

corpus = dataset['train']['text']
common_words = get_top_n_words(corpus, 30)
df = pd.DataFrame(common_words, columns=['unigram', 'count'])

fig = go.Figure([go.Bar(x=df['unigram'], y=df['count'])])
fig.update_layout(title=go.layout.Title(text="Top 30 unigrams"))
fig.show()

## Nétoyage des données (tokenizer, stop word, ponctuation, lemmatization)

In [22]:
# Data cleaning
import spacy

nlp = spacy.load("en_core_web_sm")

def clean_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])



In [ ]:
X_train = [clean_text(text) for text in dataset['train']['text']]

In [32]:
X_train[:5]

['quote Oedpius Rex tragedy film bad movie see way Uwe Boll hater like humourless people IMDb br boll movie like Postal Tunnel rat hilariously bad entertaining honestly movie horrible hate zero star rating story crap spend fifth film build plot middle scene grizzly horrible tastelessly murder finally end villan win end totally acceptable surely tasteful br Uwe Boll like say early violent movie love violent movie love see movie Hostel movie Tokyo Gore Police New York Ripper movie Dog Soldiers Bloody Valentine House Left Watchmen Wolf Creek tarantino movie Sam Peckinpah Cannibal Holocaust OMFG br cruel sadistic pervert look movie list like Cannibal Holocaust bad Uwe dark funny light hearted like Ed Wood awful experience feel horrible see WATCH avoid cost',
 'greeting darkness happen great Barry Levinson direct time favorite Avalon Diner fine movie Rainman provide interest believe bad thing comedy boring envy definition boring big fan pure slap stick dumb Dumber stun god awful movie maybe

In [26]:
X_test = [clean_text(text) for text in dataset['test']['text']]

In [27]:
y_train = dataset['train']['label']
y_test = dataset['test']['label']

In [40]:
corpus = X_train
common_words = get_top_n_words(corpus, 30)
df = pd.DataFrame(common_words, columns=['unigram', 'count'])

fig = go.Figure([go.Bar(x=df['unigram'], y=df['count'])])
fig.update_layout(title=go.layout.Title(text="Top 30 unigrams"))
fig.show()

## Entrainement

### KNN

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

# Creat simple pipline that do tfidf 
# and train Multilabel classification model with LinearSVC 
SVC_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', KNeighborsClassifier(n_neighbors=5, weights="distance")),
])

# max_features -> beaucoup de features = beaucoup de temps de calcule
# TfidfVectorizer(max_features=1000)

SVC_pipeline.fit(X_train, y_train)

# compute the testing accuracy
prediction = SVC_pipeline.predict(X_test)
print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))
print(classification_report(y_test, prediction))

Test accuracy is 0.7706666666666667
              precision    recall  f1-score   support

           0       0.80      0.72      0.76      2625
           1       0.75      0.82      0.78      2625

    accuracy                           0.77      5250
   macro avg       0.77      0.77      0.77      5250
weighted avg       0.77      0.77      0.77      5250



### KNN sans preprocessing

In [46]:
# Creat simple pipline that do tfidf 
# and train Multilabel classification model with LinearSVC 
SVC_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', KNeighborsClassifier(n_neighbors=5, weights="distance")),
])

SVC_pipeline.fit(dataset['train']['text'], dataset['train']['label'])

prediction = SVC_pipeline.predict(dataset['test']['text'])
print('Test accuracy is {}'.format(accuracy_score(dataset['test']['label'], prediction)))
print(classification_report(dataset['test']['label'], prediction))

Test accuracy is 0.7579047619047619
              precision    recall  f1-score   support

           0       0.77      0.74      0.75      2625
           1       0.75      0.78      0.76      2625

    accuracy                           0.76      5250
   macro avg       0.76      0.76      0.76      5250
weighted avg       0.76      0.76      0.76      5250



### SVM

In [41]:
# SVM
from sklearn.svm import LinearSVC

SVC_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC()),
])

SVC_pipeline.fit(X_train, y_train)

prediction = SVC_pipeline.predict(X_test)
print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))
print(classification_report(y_test, prediction))


/home/rom/Documents/MATH904/env/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning:

The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.



Test accuracy is 0.8761904761904762
              precision    recall  f1-score   support

           0       0.89      0.86      0.87      2625
           1       0.86      0.89      0.88      2625

    accuracy                           0.88      5250
   macro avg       0.88      0.88      0.88      5250
weighted avg       0.88      0.88      0.88      5250



### Naive Bayes

In [42]:
# Naive Bayes
from sklearn.naive_bayes import MultinomialNB

NB_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB()),
])

NB_pipeline.fit(X_train, y_train)

prediction = NB_pipeline.predict(X_test)
print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))
print(classification_report(y_test, prediction))

Test accuracy is 0.8632380952380952
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      2625
           1       0.86      0.86      0.86      2625

    accuracy                           0.86      5250
   macro avg       0.86      0.86      0.86      5250
weighted avg       0.86      0.86      0.86      5250



### Random Forest

In [44]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

RF_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier()),
])

RF_pipeline.fit(X_train, y_train)

prediction = RF_pipeline.predict(X_test)

print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))
print(classification_report(y_test, prediction))

Test accuracy is 0.8466666666666667
              precision    recall  f1-score   support

           0       0.85      0.85      0.85      2625
           1       0.85      0.85      0.85      2625

    accuracy                           0.85      5250
   macro avg       0.85      0.85      0.85      5250
weighted avg       0.85      0.85      0.85      5250



## Questions
1. Comment la réduction du nombre de caractéristiques (`max_features`) affecte-t-elle la performance du modèle ?  
La réduction du nombre de caractéristiques (max_features) dans un modèle, comme dans le cas des arbres de décision par exemple, peut avoir un impact sur la performance de celui-ci. En réduisant le nombre de caractéristiques, on réduit également la complexité du modèle, ce qui peut aider à prévenir le surajustement (overfitting) dans certains cas, notamment lorsque le nombre de caractéristiques est très élevé par rapport à la taille des données d'entraînement. Cependant, une réduction excessive du nombre de caractéristiques peut également entraîner une perte d'informations importantes pour la tâche de prédiction, ce qui peut affecter négativement la performance du modèle en réduisant sa capacité à généraliser sur de nouvelles données.

2. Quel impact a le choix du nombre de voisins dans KNN sur les résultats ?  
Dans l'algorithme des k plus proches voisins (KNN), le choix du nombre de voisins (k) a un impact significatif sur les résultats du modèle. Un k plus petit signifie que le modèle se base sur moins de voisins pour effectuer sa prédiction, ce qui peut le rendre plus sensible au bruit ou aux variations aléatoires dans les données. Un k plus grand peut lisser les frontières de décision, ce qui peut être bénéfique pour réduire le surajustement, mais peut également conduire à une sous-représentation des structures locales dans les données. Ainsi, le choix optimal de k dépendra du jeu de données spécifique et de la complexité de la tâche de classification. Dans mon cas la meilleur valeur est de 5 qui nous donne une accuracy de 77%

3. Comparez les performances du modèle KNN avec un autre classificateur (par exemple, [Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) ou [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC). Lequel performe mieux et pourquoi ?  
Le SVM et Naive Bayes founissent de meilleur résultats que le KNN. Les SVm sont en général meilleur pour des probèmes avec une forte dimmension des données. Naive Bayes peut être plus approprié pour des tâches de classification simples avec des hypothèses d'indépendance entre les caractéristiques

4. Le preprocessing améliore t-il la clasification ?  
Dans ce cas-ci le préprocessing n'as pas changé beaucoup les résultats car le tf idf permet de garder en général les mots important.